In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

from mis_utils import *

In [2]:
# Definiciones de constantes

node_names = np.array(('s', '2', '3', '4', '5', 't'))

# Balances: Sale desde s y llega hasta t
beq = np.array([1, 0, 0, 0, 0, -1])

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# Matrices resultantes de NN a NA
Aeq, arc_idxs = nn2na(NN, node_names = node_names, show_results = True)

# Guardo los nombres de los nodo-arco-nodo posibles
nan_names = get_col_names(NN, node_names, as_numpy=True)

# Restricciones, l <= x <= u
# Entre 0 e infinito
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

[[0 1]
 [0 2]
 [1 3]
 [1 5]
 [2 4]
 [3 5]
 [4 5]]
Input: 
 [[0 1 1 0 0 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]

Column names: s2-s3-24-2t-35-4t-5t

Output: 
 [[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]]



In [3]:
# Vector de costos
# Con el orden de los nombres de columnas, sacado de los resultados. Column names: s2-s3-24-2t-35-4t-5t
C = np.array([2, 2, 2, 5, 2, 1, 2])

In [4]:
pd.DataFrame(data=NN, columns=node_names)

,s,2,3,4,5,t
0,0,1,1,0,0,0
1,0,0,0,1,0,1
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1
5,0,0,0,0,0,0


In [5]:
pd.DataFrame(data=Aeq, columns=nan_names)

,s2,s3,24,2t,35,4t,5t
0,1,1,0,0,0,0,0
1,-1,0,1,1,0,0,0
2,0,-1,0,0,1,0,0
3,0,0,-1,0,0,1,0
4,0,0,0,0,-1,0,1
5,0,0,0,-1,0,-1,-1


In [6]:
# Resumen
print('## Optimizer inputs ## \n\n'
      'Cost vector: %s \n\n'
      'Columns: %s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s \n' % (C, nan_names, Aeq, beq, bounds))

## Optimizer inputs ## 

Cost vector: [2 2 2 5 2 1 2] 

Columns: ['s2' 's3' '24' '2t' '35' '4t' '5t'] 

A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]] 

b_eq demand-supply vector: [ 1  0  0  0  0 -1] 

Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)) 



In [7]:
len_nodes = NN.shape[0]
initial_node = 0

# Dijkstra

## Pasar a función

In [8]:
weights, prec = dijkstra(NN, C, node_names)


Sin explorar:  [0, 1, 2, 3, 4, 5]
Pesos de los nodos sin explorar:  [0.0, inf, inf, inf, inf, inf]
Nodo elegido  s
s  =>  2  con un costo de  2
s  =>  3  con un costo de  2
[ 0.  2.  2. inf inf inf]
[0 0 0 0 0 0]
Tabla de pesos:
 [[ 0.  2.  2. inf inf inf]
 [ 0.  2.  2. inf inf inf]]
Tabla de precedentes:
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]]


Sin explorar:  [1, 2, 3, 4, 5]
Pesos de los nodos sin explorar:  [2.0, 2.0, inf, inf, inf]
Nodo elegido  2
2  =>  4  con un costo de  2
2  =>  t  con un costo de  5
[ 0.  2.  2.  4. inf  7.]
[0 0 0 1 0 1]
Tabla de pesos:
 [[ 0.  2.  2. inf inf inf]
 [ 0.  2.  2. inf inf inf]
 [ 0.  2.  2.  4. inf  7.]]
Tabla de precedentes:
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 1 0 1]]


Sin explorar:  [2, 3, 4, 5]
Pesos de los nodos sin explorar:  [2.0, 4.0, inf, 7.0]
Nodo elegido  3
3  =>  5  con un costo de  2
[0. 2. 2. 4. 4. 7.]
[0 0 0 1 2 1]
Tabla de pesos:
 [[ 0.  2.  2. inf inf inf]
 [ 0.  2.  2. inf inf inf]
 [ 0.  2.  2.  4. inf  7.]
 [ 0.  2.  2.  4.  4.  

In [9]:
# Muestro los resultados

prec_actual = np.argwhere(node_names == 't')[0][0]
prec_fin = np.argwhere(node_names == 's')[0][0]

fin = False

path = np.array(())
w_path = np.array(())
while not fin:
    path = np.append(node_names[prec_actual], path)
    w_path = np.append(weights[prec_actual], w_path)
    prec_actual = prec[prec_actual]

    if prec_actual == prec_fin:
        fin = True

path = np.append(node_names[prec_actual], path)
w_path = np.append(weights[prec_actual], w_path)

In [10]:
print("Camino: ", path)

Camino:  ['s' '2' '4' 't']


In [11]:
print("Costos: ", w_path)

Costos:  [0. 2. 4. 5.]


In [12]:
set(['B', 'C'])

{'B', 'C'}

In [13]:
{'B', 'C'}

{'B', 'C'}

In [14]:
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    print(start)
    for next in graph[start] - visited:
        dfs(graph, next, visited)
    return visited

graph = {'s': set(['2', '3']),
         '2': set(['4', 't']),
         '3': set(['5']),
         '4': set(['t']),
         '5': set(['t']),
         't': set()}

dfs(graph, 's')


s
3
5
t
2
4


{'2', '3', '4', '5', 's', 't'}